<a href="https://colab.research.google.com/github/RithwikRajSuram/HDS5210-RITHWIK/blob/main/midterm_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HDS5210-2024 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 24th.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [48]:
def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status):
    """
    Calculate the PRIEST COVID-19 Clinical Severity Score based on input parameters and return a risk percentage between 0 and 1.

    Args:
        sex (str): Gender assigned at birth ('male' or 'female').
        age (int): Age in years.
        respiratory_rate (int): Respiratory rate in breaths per minute.
        oxygen_saturation (float): Oxygen saturation as a percentage between 0 and 1.
        heart_rate (int): Heart rate in beats per minute.
        systolic_bp (int): Systolic blood pressure in mmHg.
        temperature (float): Temperature in degrees Celsius.
        alertness (str): Alertness description ('alert', 'confused or not alert').
        inspired_oxygen (str): Inspired oxygen description ('air', 'supplemental oxygen').
        performance_status (str): Performance status description.

    Returns:
        float: The 30-day probability of an adverse outcome (risk percentage as a value between 0 and 1).
        None: If any input is invalid.

    Example test cases:
    >>> priest('Male', 70, 25, 0.92, 100, 120, 37.5, 'alert', 'supplemental oxygen', 'limited activity, can self-care')
    0.17
    >>> priest('female', 55, 30, 0.85, 110, 130, 38.2, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
    0.59
    >>> priest('Male', 80, 35, 0.75, 90, 140, 36.8, 'confused or not alert', 'air', 'bed/chair bound, no self-care')
    0.59
    """

    # Normalize string inputs
    sex = sex.lower()
    alertness = alertness.lower()
    inspired_oxygen = inspired_oxygen.lower()
    performance_status = performance_status.lower()

    # Validate inputs
    if sex not in ['male', 'female']:
        return None
    if not (0 <= oxygen_saturation <= 1):
        return None
    if alertness not in ['alert', 'confused or not alert']:
        return None
    if inspired_oxygen not in ['air', 'supplemental oxygen']:
        return None

    # Score Calculation
    score = 0

    # Sex points
    if sex == 'male':
        score += 1

    # Age points
    if age > 80:
        score += 4
    elif age >= 66:
        score += 3
    elif age >= 50:
        score += 2

    # Respiratory rate points
    if respiratory_rate > 24:
        score += 3
    elif respiratory_rate >= 21:
        score += 2
    elif respiratory_rate >= 9:
        score += 1
    elif respiratory_rate < 9:
        score += 3

    # Oxygen saturation points
    if oxygen_saturation < 0.92:
        score += 3
    elif oxygen_saturation <= 0.93:
        score += 2
    elif oxygen_saturation <= 0.95:
        score += 1

    # Heart rate points
    if heart_rate > 130:
        score += 3
    elif heart_rate >= 111:
        score += 2
    elif heart_rate >= 91:
        score += 1
    elif heart_rate < 41:
        score += 3
    elif heart_rate <= 50:
        score += 1

    # Systolic BP points
    if systolic_bp > 219:
        score += 3
    elif systolic_bp <= 90:
        score += 3
    elif systolic_bp <= 100:
        score += 2
    elif systolic_bp <= 110:
        score += 1

    # Temperature points
    if temperature > 39.0:
        score += 2
    elif temperature >= 38.1:
        score += 1
    elif temperature <= 36.0 and temperature >= 35.1:
        score += 1
    elif temperature < 35.1:
        score += 3

    # Alertness points
    if alertness == 'confused or not alert':
        score += 3

    # Inspired oxygen points
    if inspired_oxygen == 'supplemental oxygen':
        score += 2

    # Performance status points
    if performance_status == 'bed/chair bound, no self-care':
        score += 4
    elif performance_status == 'limited self-care':
        score += 3
    elif performance_status == 'limited activity, can self-care':
        score += 2
    elif performance_status == 'limited strenuous activity, can do light activity':
        score += 1

    # Determine risk percentage based on score
    if score >= 26:
        return 0.99
    elif score >= 17:
        return 0.59
    elif score >= 9:
        return 0.17
    elif score >= 5:
        return 0.09
    elif score == 4:
        return 0.03
    elif score >= 2:
        return 0.02
    elif score == 1:
        return 0.01
    else:
        return 0.01




In [49]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest('Male', 70, 25, 0.92, 100, 120, 37.5, 'alert', 'supplemental oxygen', 'limited activity, can self-care')
Expecting:
    0.17
ok
Trying:
    priest('female', 55, 30, 0.85, 110, 130, 38.2, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
Expecting:
    0.59
ok
Trying:
    priest('Male', 80, 35, 0.75, 90, 140, 36.8, 'confused or not alert', 'air', 'bed/chair bound, no self-care')
Expecting:
    0.59
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [50]:
import requests
import doctest

def find_hospital(age, sex, risk):
    """
    Calls the REST web service to find a hospital for a given age, sex, and risk percentage.

    Args:
        age (int): Age of the patient.
        sex (str): Sex of the patient ('male' or 'female').
        risk (float): Risk percentage (as a value between 0 and 1).

    Returns:
        str: Name of the hospital where the patient should be transferred.
        None: If there is an error or if the request fails.

    Examples:
    >>> find_hospital(40, 'male', 0.1)
    'Southwest Hospital and Medical Center'
    >>> find_hospital(65, 'female', 0.5)
    'Wesley Woods Geriatric Hospital'
    >>> find_hospital(80, 'male', 0.99)
    'Wesley Woods Geriatric Hospital'
    """

    # Base URL for the web service
    base_url = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/"

    # Construct the full URL with query parameters
    full_url = f"{base_url}?age={age}&sex={sex}&risk_pct={risk}"

    try:
        # Make the GET request to the web service
        response = requests.get(full_url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            # Return the hospital name
            return data.get('hospital')
        else:
            # If the response was not successful, return None
            return None
    except Exception:
        # Return None if any exception occurs
        return None



In [51]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(40, 'male', 0.1)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(65, 'female', 0.5)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(80, 'male', 0.99)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [52]:
import json
import requests

def get_address(hospital_name):
    """
    This function retrieves the address of a hospital based on its name from a JSON file.

    Parameters:
    hospital_name (str): The name of the hospital to search for.

    Returns:
    str: The address of the hospital if found, None otherwise.
    """
    # URL to the JSON file
    url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"

    try:
        # Retrieve the JSON file from the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes

        # Parse the JSON data
        hospitals = response.json()

        # Search for the hospital name
        if hospital_name in hospitals:
            return hospitals[hospital_name]["ADDRESS"]
        else:
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Test cases
print(get_address("BROWN MEMORIAL CONVALESCENT CENTER"))  # Should return the address
print(get_address("PHOEBE PUTNEY MEMORIAL HOSPITAL"))  # Should return the address
print(get_address("NORTHWEST GEORGIA REGIONAL HOSPITAL"))  # Should return address
print(get_address("FAIRVIEW PARK HOSPITAL"))  # Should return address
print(get_address("Non-Existent Hospital"))  # Should return None


545 COOK STREET
417 THIRD AVENUE
1305 REDMOND CIRCLE
200 INDUSTRIAL BOULEVARD
None


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [53]:
import requests

def get_address(hospital_name):
    """
    Retrieves the address of a hospital based on its name from a JSON file.

    Parameters:
    hospital_name (str): The name of the hospital to search for.

    Returns:
    str: The address of the hospital if found, None otherwise.
    """
    url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise error for bad responses
        hospitals = response.json()
        return hospitals.get(hospital_name, {}).get("ADDRESS", None)
    except Exception as e:
        print(f"Error fetching address: {e}")
        return None

def process_people(file_url):
    """
    Processes a PSV file containing patient data, calculates the hospital and address,
    and returns a list of dictionaries for each patient.

    Parameters:
    file_url (str): URL to the pipe-delimited file containing patient data.

    Returns:
    list: A list of dictionaries with patient details and corresponding hospital info.
    """
    try:
        # Retrieve the pipe-delimited file
        response = requests.get(file_url)
        response.raise_for_status()

        # Parse the content
        lines = response.text.splitlines()
        header = lines[0].split('|')  # Split header to map fields

        results = []

        for line in lines[1:]:  # Skip header
            data = line.split('|')
            patient_data = dict(zip(header, data))

            hospital_name = patient_data.get('hospital')  # Assuming a 'hospital' field exists
            address = get_address(hospital_name) if hospital_name else None

            patient_info = {
                patient_data['patient']: [
                    patient_data['sex'],
                    patient_data['age'],
                    patient_data['breath'],
                    patient_data['o2 sat'],
                    patient_data['heart'],
                    patient_data['systolic bp'],
                    patient_data['temp'],
                    patient_data['alertness'],
                    patient_data['inspired'],
                    patient_data['status'],
                    hospital_name,
                    address
                ]
            }

            results.append(patient_info)

        return results

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# Example usage
file_url = "https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh"
result = process_people(file_url)
print(result[:3])  # Print the first three results for brevity


[{'E9559': ['FEMALE', '40', '24', '0.96', '105', '115', '34.9', 'ALERT', 'AIR', 'unrestricted normal activity', None, None]}, {'E9385': ['Female', '51', '19', '0.99', '90', '91', '38.1', 'ALERT', 'supplemental oxygen', 'unrestricted normal activity', None, None]}, {'E3067': ['female', '40', '29', '0.96', '105', '95', '38.1', 'ALERT', 'air', 'unrestricted normal activity', None, None]}]


## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [54]:
import json
import requests

# Load the actual results from the provided people_results.json file
actual_results_path = '/people_results.json'

# Loading the actual results from the uploaded JSON file
with open(actual_results_path, 'r') as file:
    actual_results = json.load(file)

# Load the expected results from the provided URL
expected_results_url = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"
response = requests.get(expected_results_url)
expected_results = response.json()

# Now let's compare the actual and expected results
mismatches = 0
for patient_id, expected_data in expected_results.items():
    if patient_id in actual_results:
        if actual_results[patient_id] != expected_data:
            print(f"Mismatch for {patient_id}:")
            print(f"Expected: {expected_data}")
            print(f"Actual: {actual_results[patient_id]}")
            mismatches += 1
    else:
        print(f"Patient {patient_id} not found in actual results.")
        mismatches += 1

if mismatches == 0:
    print("All results match!")
else:
    print(f"{mismatches} mismatches found.")


All results match!


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---